## Sentiment data preparation and file creation 

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore

In [4]:
sns_files = pd.read_csv(r'C:\Users\B420615.STANLIB\SNS_Control.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\B420615.STANLIB\\SNS_Control.csv'

In [ ]:
sns_files = sns_files['Files'].tolist()

In [ ]:
#create two lists of desired column names - one for all columns and one for data columns
adjcols = ['Date','Ticker','Name','BM','AdjfcfY','CECP','TR_1M','Price','FCFY','MCap','TR_12-1','best_eps_chg','rec_chg12M','tp_chg','fcf_me','fcf_at','seas_1_1an','rec_chg']
data_cols = ['BM','AdjfcfY','CECP','TR_1M','Price','FCFY','MCap','TR_12-1','best_eps_chg','rec_chg12M','tp_chg','fcf_me','fcf_at','seas_1_1an','rec_chg']
cols_to_avg = ['BM','AdjfcfY','CECP','TR_1M','Price','FCFY','MCap','TR_12-1','best_eps_chg','rec_chg12M','tp_chg','fcf_me','fcf_at']
excl_cols = ['seas_1_1an','rec_chg']

In [ ]:
for file in sns_files[:-1]:

    try:
        sentiment_df = pd.read_csv(rf'C:\Users\B420615.STANLIB\OneDrive - Standard Bank\Documents\data\Neg_Sel\{file}.csv', thousands=',')


        # clean data so that all values, apart from naming values, and including NaNs, are reflected as float values. Delete unneeded column
        sentiment_df = sentiment_df[sentiment_df['Ticker'].notna()]
        sentiment_df.replace('N.A.', np.nan, inplace=True)
        sentiment_df.drop(['EPS_ESTIMATE_CHG'], axis=1,inplace=True)
        sentiment_df.columns=adjcols

        def convert_to_float(value):
            if isinstance(value, str):  # Check if the value is a string
                value = value.replace(',', '')  # Remove commas
                try:
                    return float(value)  # Convert to float
                except ValueError:
                    return None  # Return None if conversion fails
            return value  # Return the value unchanged if not a string

        sentiment_df[data_cols]=sentiment_df[data_cols].applymap(convert_to_float)


        for col in cols_to_avg:
            if col in sentiment_df.columns:
                mean_value = sentiment_df[col].astype(float).mean()
                sentiment_df[col].fillna(mean_value,inplace=True)

        #calculate z-score for all cols where  mean values are imputed and append to existing data frame
        z_scores = sentiment_df.iloc[:, 3:-2].apply(zscore)
        z_scores.columns = [f'z_{col}' for col in sentiment_df.columns[3:-2]]
        sentiment_df = pd.concat([sentiment_df, z_scores], axis=1)


        excl_stocks = []
        for index,row in sentiment_df.iterrows():
            if pd.isna(row['seas_1_1an']) or pd.isna(row['rec_chg']):
                excl_stocks.append(row['Ticker'])

        # drop na columns where na cols are in excl_cols
        sentiment_df = sentiment_df.dropna(subset = ['seas_1_1an','rec_chg'])

        #calculate z scores for excl_cols
        z_scores = sentiment_df[excl_cols].apply(zscore)
        z_scores.columns = ['z_seas_1_1an','z_rec_chg']

        # add z-score columns to the original DataFrame
        sentiment_df = pd.concat([sentiment_df, z_scores], axis=1)
        
        #calculate z-scores using specified weights for diff variables. Note sign for z_TR_1M is negative (lower is better)
        sentiment_df['Sentiment'] = (sentiment_df['z_CECP']*0.2)+(sentiment_df['z_best_eps_chg']*0.2)+(sentiment_df['z_tp_chg']*0.2)+(sentiment_df['z_rec_chg']*0.2)+(sentiment_df['z_seas_1_1an']*0.1)+(sentiment_df['z_TR_1M']*-0.1)
        #columns_for_Sentiment = ['z_TR_12-1', 'z_CECP', 'z_best_eps_chg','z_rec_chg','z_tp_chg','z_TR_1M','z_seas_1_1an']
        #sentiment_df['Sentiment'] = sentiment_df[columns_for_Sentiment].mean(axis=1)

        
        # Divide universe into quintiles
        sentiment_df['S_Quintile'] = pd.qcut(sentiment_df['Sentiment'], q=5, labels=False)+1
        
        #Save processed sentiment csv file
        output_file_path = rf'C:\Users\B420615.STANLIB\OneDrive - Standard Bank\Documents\data\Neg_Sel\Sentiment_S{file}.csv'
        sentiment_df.to_csv(output_file_path, index=False)
        
    except Exception as e:
        print(f"Error processing file {file}: {e}")         